<a href="https://colab.research.google.com/github/Vlad-Shapar/machine-learning/blob/main/%D0%A8%D0%B0%D0%BF%D0%B0%D1%80%D1%8C_%D0%BF%D1%80_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357280 sha256=ccb58b5317aac5c316e24562087246ca27005dd5ce710537692f88d0d337e3be
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
'''Завдання 1.
1. Завантажте датасет для рецензій (ml-100k) за допомогою бібліотеки
Surprise.'''
from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split

data = Dataset.load_builtin("ml-100k")

trainset, testset = train_test_split(data, test_size=0.25)
algo = SVD()

algo.fit(trainset)
predictions = algo.test(testset)

accuracy.rmse(predictions)


Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
RMSE: 0.9413


0.9412872334079756

In [3]:
'''2. Виведіть перші 5 рядків завантаженого датасету.'''
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split

data = Dataset.load_builtin('ml-100k')

print("Перші 5 рядків датасету:")
for row in data.raw_ratings[:5]:
    user_id, movie_id, rating, timestamp = row
    print(f"Користувач {user_id} оцінив фільм {movie_id} на {rating} балів (часова позначка: {timestamp})")

Перші 5 рядків датасету:
Користувач 196 оцінив фільм 242 на 3.0 балів (часова позначка: 881250949)
Користувач 186 оцінив фільм 302 на 3.0 балів (часова позначка: 891717742)
Користувач 22 оцінив фільм 377 на 1.0 балів (часова позначка: 878887116)
Користувач 244 оцінив фільм 51 на 2.0 балів (часова позначка: 880606923)
Користувач 166 оцінив фільм 346 на 1.0 балів (часова позначка: 886397596)


In [4]:
'''3. Реалізуйте два алгоритми для рекомендаційної системи на основі цього
датасету. Можна вибрати будь-які алгоритми з бібліотеки Surprise.
4. Використайте крос-валідацію для підбору оптимальних параметрів для
обох алгоритмів.'''
trainset, testset = train_test_split(data, test_size=0.2)

# Параметри для підбору через GridSearchCV для SVD
param_grid_svd = {
    'n_factors': [25, 100],
    'lr_all': [0.005, 0.015],
    'reg_all': [0.02, 0.05]
}

# Параметри для підбору через GridSearchCV для KNNBasic
param_grid_knn = {
    'k': [20, 40],
    'min_k': [1, 3],
    'sim_options': {
        'name': ['cosine', 'pearson'],
        'user_based': [False, True]
    }
}

grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5)
grid_search_svd.fit(data)

grid_search_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['rmse', 'mae'], cv=3)
grid_search_knn.fit(data)



Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Comput

In [5]:
'''5. Оберіть найкращий алгоритм на основі середньої абсолютної помилки
(MAE).'''

best_svd = grid_search_svd.best_estimator['mae']
best_knn = grid_search_knn.best_estimator['mae']

print(f"Найкращий MAE для SVD: {grid_search_svd.best_score['mae']}")
print(f"Найкращий MAE для KNNBasic: {grid_search_knn.best_score['mae']}")

# Вибір моделі з найменшим MAE
if grid_search_svd.best_score['mae'] < grid_search_knn.best_score['mae']:
    best_model = best_svd
    print("Вибраний алгоритм: SVD")
else:
    best_model = best_knn
    print("Вибраний алгоритм: KNNBasic")


Найкращий MAE для SVD: 0.7288065397002967
Найкращий MAE для KNNBasic: 0.809324447049284
Вибраний алгоритм: SVD


In [6]:
'''6. Виведи рекомендації (10 фільмів) для конкретного користувача.'''
best_model.fit(trainset)

# Виведення рекомендацій для конкретного користувача
user_id = str(75)
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print(f"Кількість рецензій користувача {user_id}: {len(user_ratings)}")

# Отримання рекомендацій
all_items = set(trainset.all_items())
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

# Прогнозування рейтингів для нерецензованих фільмів
predictions = [(item, best_model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

# Виведення 10 найкращих фільмів
print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")

Кількість рецензій користувача 75: 66
Топ-10 фільмів, рекомендованих для користувача:
Фільм 474 з прогнозованим рейтингом 5.00
Фільм 185 з прогнозованим рейтингом 5.00
Фільм 318 з прогнозованим рейтингом 5.00
Фільм 603 з прогнозованим рейтингом 4.92
Фільм 429 з прогнозованим рейтингом 4.80
Фільм 59 з прогнозованим рейтингом 4.76
Фільм 423 з прогнозованим рейтингом 4.74
Фільм 20 з прогнозованим рейтингом 4.73
Фільм 1143 з прогнозованим рейтингом 4.73
Фільм 480 з прогнозованим рейтингом 4.73


In [7]:
'''Завдання 2.
1. Побудуйте власну рекомендаційну систему (приклад є в документації).
Оцініть її.'''
from surprise import NMF, NormalPredictor
from surprise.model_selection import cross_validate

algo_nmf = NMF()

algo_random = NormalPredictor()

cv_nmf = cross_validate(algo_nmf, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

cv_random = cross_validate(algo_random, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

print(f"Результати для NMF:\nRMSE: {cv_nmf['test_rmse'].mean()}\nMAE: {cv_nmf['test_mae'].mean()}")
print(f"Результати для випадкового прогнозу:\nRMSE: {cv_random['test_rmse'].mean()}\nMAE: {cv_random['test_mae'].mean()}")


Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9656  0.9660  0.9716  0.9560  0.9650  0.9648  0.0050  
MAE (testset)     0.7593  0.7592  0.7641  0.7507  0.7593  0.7585  0.0043  
Fit time          1.90    1.97    2.35    3.93    2.03    2.43    0.76    
Test time         0.10    0.24    0.21    0.10    0.11    0.15    0.06    
Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5212  1.5171  1.5214  1.5088  1.5128  1.5162  0.0049  
MAE (testset)     1.2209  1.2228  1.2229  1.2154  1.2121  1.2188  0.0043  
Fit time          0.10    0.12    0.12    0.17    0.12    0.13    0.02    
Test time         0.08    0.20    0.08    0.24    0.08    0.14    0.07    
Результати для NMF:
RMSE: 0.9648324251432239
MAE: 0.7585077082522316
Результати для випадкового прогнозу:
RMSE: 1.5162423037570087

In [8]:
'''2. Отримайте рекомендацію для певного користувача.'''
algo_nmf.fit(trainset)

user_id = str(75)
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]

unrated_items = all_items - rated_items
predictions = [(item, algo_nmf.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

print(f"Топ-10 рекомендацій для користувача {user_id}:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")


Топ-10 рекомендацій для користувача 75:
Фільм 170 з прогнозованим рейтингом 5.00
Фільм 483 з прогнозованим рейтингом 5.00
Фільм 1591 з прогнозованим рейтингом 5.00
Фільм 1397 з прогнозованим рейтингом 5.00
Фільм 1021 з прогнозованим рейтингом 5.00
Фільм 958 з прогнозованим рейтингом 5.00
Фільм 1039 з прогнозованим рейтингом 5.00
Фільм 1019 з прогнозованим рейтингом 5.00
Фільм 169 з прогнозованим рейтингом 5.00
Фільм 1598 з прогнозованим рейтингом 5.00


In [ ]:
'''У ході лабораторної роботи було вивчено рекомендаційні системи за допомогою
бібліотеки Surprise. Ми використали датасет рецензій (ml-100k) і реалізували
алгоритми SVD та KNNBasic, провівши крос-валідацію для підбору оптимальних
параметрів. Також була побудована рекомендаційна система на
основі алгоритму NMF, результати якої порівняли з випадковим прогнозуванням.
Метод факторизації матриць продемонстрував свою ефективність. '''